In [1]:
import nltk
import pickle
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
with open('pengetahuan.json') as json_data:
    intents = json.load(json_data)

words = []
classes = []
documents = []
ignore_words = ['?']

In [3]:
for intent in intents['pengetahuan']:
    for pattern in intent['patterns']:
        # tokenize setiap kata didalam kalimat
        w = nltk.word_tokenize(pattern)
        # tambahkan kedalam list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print(documents, "documents")
print(classes, "classes")
print("unique stemmed words", words)

[(['Hi'], 'salam'), (['halo'], 'salam'), (['pagi'], 'salam'), (['sewa', 'dong'], 'salam'), (['ada', 'orang', 'disana', '?'], 'salam'), (['halo'], 'salam'), (['bos'], 'salam'), (['bang'], 'salam'), (['test'], 'salam'), (['mau', 'tanya', 'dong'], 'salam'), (['Bye'], 'berpisah'), (['Sampai', 'Ketemu', 'nanti'], 'berpisah'), (['Sampai', 'nanti'], 'berpisah'), (['dada'], 'berpisah'), (['makasih'], 'berpisah'), (['thanks'], 'berpisah'), (['Makasih'], 'makasih'), (['Terimakasih'], 'makasih'), (['Sangat', 'Membantu'], 'makasih')] documents
['berpisah', 'makasih', 'salam'] classes
unique stemmed words ['ada', 'bang', 'bantu', 'bos', 'bye', 'dada', 'dong', 'halo', 'hi', 'ketemu', 'makasih', 'mau', 'nanti', 'orang', 'pagi', 'sampai', 'sana', 'sangat', 'sewa', 'tanya', 'terimakasih', 'test', 'thanks']


In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [6]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [7]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
input_h = tflearn.input_data(shape=(None, len(train_x[0])))
h2 = tflearn.fully_connected(input_h, 9)
h3 = tflearn.fully_connected(h2, 18)
h4 = tflearn.fully_connected(h3, 18)
h5 = tflearn.fully_connected(h4, 9)

output_h = tflearn.fully_connected(h5, len(train_y[0]), activation='softmax')
output_h_reg = tflearn.regression(output_h)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# Define model and setup tensorboard
model = tflearn.DNN(output_h_reg, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=5000, batch_size=10, show_metric=True)
model.save('model.tflearn')

Training Step: 9999  | total loss: 0.17004 | time: 0.003s
| Adam | epoch: 5000 | loss: 0.17004 - acc: 0.9063 -- iter: 10/19
Training Step: 10000  | total loss: 0.15663 | time: 0.007s
| Adam | epoch: 5000 | loss: 0.15663 - acc: 0.9157 -- iter: 19/19
--
INFO:tensorflow:C:\xampp\htdocs\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words(disingkat bow) array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("test: %s" % w)

    return np.array(bag)

pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))